# Treatment of Covid-19 with Remdesivir - Systematic Review and Meta-Analysis

## Search pubmed for appropriate studies
1. Click on this [link](https://pubmed.ncbi.nlm.nih.gov/)
2. Type "(((remdesivir) AND (covid)) NOT (review)) NOT (meta)" into search box
3. On the right hand side check "Free Full Text"
4. Click on save and select all results in selection and format Pubmed
5. Click on create file and save it under the name remdesivir.txt

## Filter for Randomized Control Trials
We only want to include randomized control trials in our meta-analysis. We are using a machine learning tool called robotsearch to filter out only the studies from the pubmed search which used a ranodmized control trial design.
1. Move the file into the robotsearch directory
2. Setup virtual environment for robotsearch
    1. To use as a Python package you will need an install of conda. [Miniconda](https://docs.conda.io/en/latest/miniconda.html) is a good choice.
    2. `cd` your way into the cloned repo.
    3. Run `conda env create -f rs_env.yml` to create the environment with the required packages.
    4. Activate the conda environment `conda activate robotsearch`.
    5. Run `python setup.py install`.
3. Run `robotsearch remdesivir.txt` 

The results are saved in the file remdesivir_robotviewer_RCTs.txt. Let's look at the result

In [40]:
from robotsearch.parsers import ris
file_input = "robotsearch/remdesivir.txt"
file_result = "robotsearch/remdesivir_robotreviewer_RCTs.txt"
with open(file_input, 'r', encoding="utf8") as f:
    inp = ris.load(f)
with open(file_result, 'r', encoding="utf8") as f:
    result = ris.load(f)
print("The inital search result has {} articles".format(len(inp)))
print("{} articles were classified as rcts".format(len(result)))

other non numbered
other non numbered
The inital search result has 1847 articles
295 articles were classified as rcts


In [196]:
#extract PMID - they are found in the 'PMID' key.
pmcs = []
for i in range(len(result)):
    if "PMC" in result[i].keys():
        id_raw = result[i]['PMC'][0]
        pmc = id_raw.strip()
        pmcs.append(pmc)
len(pmcs)

286

Let's reduce the number even further by checking if the article contains necessary information. Here we only want articles that use time to clinical improvement. The article has to include the meassure hazard ratio

In [291]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
site= "https://www.ncbi.nlm.nih.gov/pmc/articles/{}/"
hdr = {'User-Agent': 'Mozilla/5.0'}
regex_time = r"method.+time to clinical improvement" # the outcome meassure must come after the word method, so it is like mentioned in the methods or results section
regex_hr = r"method.+hazard[\s-]ratio"
#pmcs_outcome = []
for i in range(250,len(pmcs)):
    print(i)
    url = site.format(pmcs[i])
    req = Request(url,headers=hdr)
    try:
        page = urlopen(req)
        soup = BeautifulSoup(page)
        if re.search(regex_time, soup.prettify(), re.DOTALL|re.IGNORECASE) and re.search(regex_hr, soup.prettify(), re.DOTALL|re.IGNORECASE):
            print(True)
            pmcs_outcome.append(pmcs[i])
    except HTTPError:
        print("httperror")

250
251
252
253
254
True
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285


In [293]:
print("{} articles contain time to clinical improvement and hazard ratio".format(len(pmcs_outcome)))
with open("pmcs.txt", "w") as f:
    for pmc in pmcs_outcome:
        f.write("%s\n" % pmc)

25 articles contain time to clinical improvement and hazard ratio


## Manual Screening of Articles
Now that we filtered out the promising studies, we must manually check for their eligibility. Even though we automated the classification as randomized controll trial, we must check for errors. We follow the following protocol
### Randomized Controlled Trial
Randomized Clinical Trials are those in which patients are assigned randomly to a group receiving treatment with Remdesivir and a placebo-group. Check the abstract and methods secion to check if the papers are randomized control trials. 

In [ ]:
## Adults only. Check min age < 18
## Participants infected with Covid-19 -> No prophylactic study
## Remdesivir only -> No combination study
## Randomized-placebo-controlled
## Time to clinical improvement + hazard ratio reported